In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.metro.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
26


In [3]:
# file_path = 'metro_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [4]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [5]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [6]:
df.query("id == 812")


,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id
4,812,Fresas y Otros Berries,False,https://www.metro.pe/frutas-y-verduras/frutas/...,0,"Fresas y Otros Frutos Rojos - Arándanos, Moras...",Descubre una variedad de fresas y frutos rojos...,801


In [7]:
file_name_categories_csv = f'metro_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [8]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)
print(len(cats))

[800, 1200, 1400, 1700, 1800, 1900, 2000, 2100, 2200, 2301, 1000003, 1000069, 1000131, 1000143, 1000144, 1001074, 1001253, 1001327, 1001374, 1001402, 1001436, 1001467, 1001470, 1001701]
24


In [9]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.metro.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 800, start = 500, end = 549
status_code = 200
cat = 1200, start = 400, end = 449
status_code = 500
cat = 1400, start = 350, end = 399
status_code = 500
cat = 1700, start = 1300, end = 1349
status_code = 200
cat = 1800, start = 600, end = 649
status_code = 500
cat = 1900, start = 200, end = 249
status_code = 500
cat = 2000, start = 800, end = 849
status_code = 200
cat = 2100, start = 1350, end = 1399
status_code = 500
cat = 2200, start = 2300, end = 2349
status_code = 200
cat = 2301, start = 1900, end = 1949
status_code = 200
cat = 1000003, start = 400, end = 449
status_code = 200
cat = 1000069, start = 400, end = 449
status_code = 500
cat = 1000131, start = 950, end = 999
status_code = 200
cat = 1000143, start = 850, end = 899
status_code = 200
cat = 1000144, start = 300, end = 349
status_code = 500
cat = 1001074, start = 1450, end = 1499
status_code = 500
cat = 1001253, start = 850, end = 899
status_code = 200
cat = 1001327, start = 550, end = 599
status_code =

In [10]:
print(len(products))

18346


In [11]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [12]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

260.0
2100-01-01T00:00:00Z
260.0
260.0


In [13]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "1031254",
    "productName": "Detergente L\u00edquido MAS Color 3L",
    "brand": "MAS Color",
    "brandId": 2004494,
    "brandImageUrl": null,
    "linkText": "detergente-liquido-mas-color-3l-1012818",
    "productReference": "1012818",
    "productReferenceCode": "1012818",
    "categoryId": "1000715",
    "productTitle": "Detergente L\u00edquido MAS Color 3L",
    "metaTagDescription": "\u00a1Encuentra aqu\u00ed los mejores productos de Detergente L\u00edquido de la marca MAS COLOR!",
    "releaseDate": "2024-09-27T00:00:00Z",
    "clusterHighlights": {
        "323": "Todo Supermercado Metro IO",
        "351": "Todo supermercado 2.0",
        "1490": "Coleccion todo Metro",
        "1917": "Coleccion todo Metro II",
        "1953": "Todo supermercado 3",
        "2020": "Todo supermercado 4.0",
        "2143": "Coleccion cupon Claro",
        "2145": "Coleccion cupon Claro diciembre",
        "2528": "compra hoy recibe hoy",
        "3402": "Todo supermercado

In [14]:
# print(json.dumps(products, indent=4))

In [15]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [16]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [17]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [18]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

print(len(products))

productId = 939171
productName = Fresa Deshidratada Natifrut x kg
brand = Genérico
brandId = 9619
brandImageUrl = None
categoryId = 1001317
productTitle = Fresa Deshidratada Natifrut x kg
metaTagDescription = ¡Encuentra aquí los mejores productos de Frutos Secos a Granel de la marca GENÉRICO!
categories = ['/Frutas y Verduras/Frutas/Frutos Secos/', '/Frutas y Verduras/Frutas/', '/Frutas y Verduras/']
categoriesIds = ['/800/801/1001317/', '/800/801/', '/800/']
link = https://www.metro.pe/fresa-deshidratada-natifrut-x-kg-771046/p
Vendido por = None
Descripción del producto = None
Presentación = None
Presentación/Empaque = None
Pack = None
Porción Sugerida = None
Azúcares Por Porción = None
Carbohidratos Por Porción = None
Descripción Nutricional = None
Grasas Por Porción = None
Advertencias de consumo = None
itemId = 39179919
ean = 22392
imageUrl = https://metroio.vteximg.com.br/arquivos/ids/518047/771046-01-56790.jpg?v=638472009850800000
Value = 260.0
IsAvailable = True
AvailableQuantit

In [19]:
i=0
for product in products:
    i+=1
    if i % 100 == 0:
        print("Iteración #: ",i)
    new_product = get_product_data(product, fecha_hora_para_base_de_datos)
    df.loc[len(df)] = new_product


Iteración #:  100
Iteración #:  200
Iteración #:  300
Iteración #:  400
Iteración #:  500
Iteración #:  600
Iteración #:  700
Iteración #:  800
Iteración #:  900
Iteración #:  1000
Iteración #:  1100
Iteración #:  1200
Iteración #:  1300
Iteración #:  1400
Iteración #:  1500
Iteración #:  1600
Iteración #:  1700
Iteración #:  1800
Iteración #:  1900
Iteración #:  2000
Iteración #:  2100
Iteración #:  2200
Iteración #:  2300
Iteración #:  2400
Iteración #:  2500
Iteración #:  2600
Iteración #:  2700
Iteración #:  2800
Iteración #:  2900
Iteración #:  3000
Iteración #:  3100
Iteración #:  3200
Iteración #:  3300
Iteración #:  3400
Iteración #:  3500
Iteración #:  3600
Iteración #:  3700
Iteración #:  3800
Iteración #:  3900
Iteración #:  4000
Iteración #:  4100
Iteración #:  4200
Iteración #:  4300
Iteración #:  4400
Iteración #:  4500
Iteración #:  4600
Iteración #:  4700
Iteración #:  4800
Iteración #:  4900
Iteración #:  5000
Iteración #:  5100
Iteración #:  5200
Iteración #:  5300
It

In [20]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
1208,1043392,Pack Abarrotes 2,Metro,2000891,None,1000671,Pack Abarrotes 2,¡Encuentra aquí los mejores productos de Aceit...,"[/Abarrotes/Aceites/Aceites Vegetales/, /Abarr...","[/1700/1709/1000671/, /1700/1709/, /1700/]",...,2050044348956,https://metroio.vteximg.com.br/arquivos/ids/53...,73.29,True,99999,73.29,73.29,73.29,2100-01-01T00:00:00Z,2024-11-13 13:02:45
13985,1022108,Mantequilla de Frutos Secos Choco Almendras Th...,The Butter Factory,2002533,None,1001269,None,None,[/Desayuno/Mermeladas y Mieles/Mantequilla de ...,"[/1001253/1001266/1001269/, /1001253/1001266/,...",...,7757505000068,https://metroio.vteximg.com.br/arquivos/ids/48...,31.50,True,100,31.50,34.90,31.50,2100-01-01T00:00:00Z,2024-11-13 13:02:45
8784,1026741,Blist Boligrafo Artesco Tops 505 3un,Artesco,3889,None,1000875,Blist Boligrafo Artesco Tops 505 3un,¡Encuentra aquí la mejor variedad en productos...,[/Libros y Librería/Útiles Escolares y Oficina...,"[/2301/1000873/1000875/, /2301/1000873/, /2301/]",...,7750082136168,https://metroio.vteximg.com.br/arquivos/ids/50...,6.20,True,10,6.20,6.20,6.20,2100-01-01T00:00:00Z,2024-11-13 13:02:45
11321,570216,Almohada de Lactancia Doble Maternelle Celeste...,Maternelle,2000468,None,1000729,Almohada de Lactancia Doble Maternelle Celeste...,¡Encuentra aquí los mejores productos de Mater...,"[/Niños y Bebés/Mundo Bebé/Maternidad/, /Niños...","[/1000143/1000169/1000729/, /1000143/1000169/,...",...,7758927006942,https://metroio.vteximg.com.br/arquivos/ids/53...,179.90,True,10,179.90,179.90,179.90,2025-09-02T00:00:00Z,2024-11-13 13:02:45
15555,573796,Pan de Yema Metro 8un,Metro,2000891,None,1001655,Pan de Yema Metro 8un,¡Encuentra aquí los mejores productos de Panes...,[/Panadería y Pastelería/La Panadería/Panes Em...,"[/1001374/1001375/1001655/, /1001374/1001375/,...",...,2050043808178,https://metroio.vteximg.com.br/arquivos/ids/52...,6.59,True,10,6.59,6.59,6.59,2100-01-01T00:00:00Z,2024-11-13 13:02:45


In [21]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
2779,1060489,"REHIDRATANTE SUEROX 630ML , FRUTOS ROJOS",Suerox,1808,https://www.metro.pe/rehidratante-suerox-630ml...,6.50,100
10850,576758,Decohome Centro de Entretenimiento Turquía Rob...,Decohome,1315,https://www.metro.pe/decohome-centro-de-entret...,1288.56,100
18074,1031468,Cartel Krea Santa Auto Viaje Navidad 2024,Krea,1001494,https://www.metro.pe/cartel-krea-santa-auto-vi...,7.20,100
12274,573584,Hp Cartucho de Tinta HP667 Tricolor,HP,1000028,https://www.metro.pe/hp-cartucho-de-tinta-hp66...,50.00,100
211,39964,Tuna Roja Metro x kg,Genérico,813,https://www.metro.pe/tuna-roja-metro-x-kg-2/p,7.99,100
1210,436,Aceite de Oliva La Española Extra Virgen Botel...,La Española,1733,https://www.metro.pe/aceite-de-oliva-la-espano...,72.50,100
4215,1023417,Gin Tanqueray Dark Berry Royale Botella 700ml ...,Tanqueray,2107,https://www.metro.pe/gin-tanqueray-dark-berry-...,89.90,10
6687,1028198,Body Lotion Beauty Care Love Summer 250ml,Beauty Care,1001211,https://www.metro.pe/body-lotion-beauty-care-l...,19.90,100


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18346 entries, 0 to 18345
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  18346 non-null  object 
 1   productName                18346 non-null  object 
 2   brand                      18346 non-null  object 
 3   brandId                    18346 non-null  int64  
 4   brandImageUrl              0 non-null      object 
 5   categoryId                 18346 non-null  object 
 6   productTitle               14520 non-null  object 
 7   metaTagDescription         11838 non-null  object 
 8   categories                 18346 non-null  object 
 9   categoriesIds              18346 non-null  object 
 10  link                       18346 non-null  object 
 11  Vendido por                0 non-null      object 
 12  Descripción del producto   18346 non-null  object 
 13  Presentación               72 non-null     object 


In [23]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.metro.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [24]:
df['product_uri_json'].sample(n=5)

10035    https://www.metro.pe/api/catalog_system/pub/pr...
17005    https://www.metro.pe/api/catalog_system/pub/pr...
9414     https://www.metro.pe/api/catalog_system/pub/pr...
14468    https://www.metro.pe/api/catalog_system/pub/pr...
10307    https://www.metro.pe/api/catalog_system/pub/pr...
Name: product_uri_json, dtype: object

In [25]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [27]:
df.iloc[18000]

productId                                                              1026717
productName                  Bolsa de Regalo Benji Santa Reno Pingüino en T...
brand                                                                    Benji
brandId                                                                   3807
brandImageUrl                                                             None
categoryId                                                             1001493
productTitle                 Bolsa de Regalo Benji Santa Reno Pingüino en T...
metaTagDescription           ¡Encuentra aquí la mejor variedad en productos...
categories                   [/Navidad/Navidad Textil/Bolsas Navideñas/, /N...
categoriesIds                [/1001470/1001476/1001493/, /1001470/1001476/,...
link                         https://www.metro.pe/bolsa-de-regalo-benji-san...
Vendido por                                                               None
Descripción del producto                            